<a href="https://colab.research.google.com/github/anshupandey/MSA-analytics/blob/main/MEM/Lab1_3_Mixed_Effect_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Module 9 – Mixed Effect Models (Labs 1–2)

**Dataset:** `Ocean_Hull_Insurance_datasetv2.csv` (provided)  
**Industry:** Ocean Hull Vessel Insurance

This notebook contains the first three labs:

- **Lab 1:** Exploring Fixed vs. Random Effects (OLS vs. Mixed-Effects)
- **Lab 2:** Frequentist vs. Bayesian (template + fallback)

> ⚠️ *Note:* The provided dataset is a cross-section (one row per vessel). It **does not contain time/longitudinal repeats** per vessel. We therefore demonstrate *hierarchical* mixed-effects with group-level (Operating Zone) random effects and random slopes. For true longitudinal modelling, add a time column (e.g., policy year per vessel) and follow the template in Lab 2 (Longitudinal Note).


In [ ]:

# ==== Setup ====
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pathlib import Path

# Matplotlib settings (per teaching guidelines: single-plot charts, no custom colors/styles)
%matplotlib inline

# Paths
DATA_PATH = "https://raw.githubusercontent.com/anshupandey/MSA-analytics/refs/heads/main/datasets/Ocean_Hull_Insurance_datasetv2.csv"

# Load data
df = pd.read_csv(DATA_PATH)

print('Shape:', df.shape)
print('Columns:', df.columns.tolist())

# Quick peek
df.head()



## Data Processing (Used in all labs)

We will:
1. Inspect missing values.
2. Engineer a **log-claim** outcome (skew reduction): `log_claim = log(Claim_Amount + 1)`.
3. Keep key predictors (fixed effects) commonly used in insurance risk:
   - `Vessel_Age`, `Premium`, `Sum_Insured`
   - Categorical risk drivers: `Vessel_Type`, `Inspection_Status`, `Weather_Risk`, `Piracy_Risk`
4. Use **Operating_Zone** as the *grouping level* for random effects to capture unobserved spatial/operational heterogeneity.


In [ ]:

# ==== Data Processing & Checks ====

# Basic NA check
na_summary = df.isna().sum().sort_values(ascending=False)
print('Missing values by column:\n', na_summary)


Missing values by column:
 Vessel_ID             0
Vessel_Type           0
Operating_Zone        0
Vessel_Age            0
Flag_State            0
Inspection_Status     0
Weather_Risk          0
Piracy_Risk           0
Claim_Occurred        0
Claim_Amount          0
Premium               0
Sum_Insured           0
Daily_Return          0
Initial_Investment    0
Final_Value           0
dtype: int64


In [ ]:
# Create target for continuous modelling
df['log_claim'] = np.log1p(df['Claim_Amount'].astype(float))

# Basic sanity checks
print('\nClaim_Amount zero rate:', (df['Claim_Amount'] == 0).mean())
print('log_claim summary:\n', df['log_claim'].describe())




Claim_Amount zero rate: 0.7666666666666667
log_claim summary:
 count    300.000000
mean       3.370222
std        6.122520
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max       15.418180
Name: log_claim, dtype: float64


In [ ]:
# Ensure categorical dtypes for formula handling
categorical_cols = ['Vessel_Type', 'Operating_Zone', 'Flag_State',
                    'Inspection_Status', 'Weather_Risk', 'Piracy_Risk']
for c in categorical_cols:
    df[c] = df[c].astype('category')


# Keep a modelling frame
model_cols = ['log_claim', 'Vessel_Age', 'Premium', 'Sum_Insured',
              'Vessel_Type', 'Inspection_Status', 'Weather_Risk', 'Piracy_Risk', 'Operating_Zone']
work = df[model_cols].dropna().copy()

print('\nModelling frame shape:', work.shape)
work.head()


Modelling frame shape: (300, 9)


,log_claim,Vessel_Age,Premium,Sum_Insured,Vessel_Type,Inspection_Status,Weather_Risk,Piracy_Risk,Operating_Zone
0,0.0,19,159500,14050000,Container Ship,Overdue,Moderate,Moderate,Strait of Malacca
1,0.0,40,170000,13000000,Container Ship,Up-to-date,High,High,Strait of Malacca
2,0.0,26,193000,18700000,Tanker,Delayed,Low,Low,Indian Ocean
3,0.0,7,123500,11650000,Bulk Carrier,Up-to-date,Moderate,Low,South China Sea
4,0.0,18,109000,7100000,Offshore Support Vessel,Delayed,Low,Low,Mediterranean



### Quick EDA

We’ll examine:
- Distribution of **log_claim**
- Average claims by **Operating_Zone** (group heterogeneity glance)


In [ ]:

# Distribution of log_claim
plt.figure()
work['log_claim'].plot.hist(bins=30)
plt.title('Distribution of log_claim')
plt.xlabel('log(Claim_Amount + 1)')
plt.ylabel('Count')
plt.show()

# Zone-level mean log_claim
zone_means = work.groupby('Operating_Zone')['log_claim'].mean().sort_values(ascending=False)
print(zone_means.head(10))



---
## Lab 1 – Exploring Fixed vs. Random Effects

**Goal:** Understand when a variable should be treated as fixed or random.  
We compare an **OLS (fixed-effects-only)** model vs. a **Mixed-Effects** model with **random intercepts by Operating_Zone**.


In [ ]:
import numpy as np, pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import patsy


In [ ]:
tmp = work.copy()

In [ ]:
# 0) Keep only zones with >= 2 observations (helps RE estimation)
zone_counts = tmp['Operating_Zone'].value_counts()
tmp = tmp[tmp['Operating_Zone'].isin(zone_counts[zone_counts >= 2].index)].copy()
tmp['Operating_Zone'] = tmp['Operating_Zone'].astype('category')

In [ ]:
# 1) Collapse rare categorical levels (min 2 rows)
def collapse_rare(series, min_count=2, other='Other'):
    vc = series.value_counts()
    rare = vc[vc < min_count].index
    return series.apply(lambda x: other if x in rare else x).astype('category')

for cat in ['Vessel_Type','Inspection_Status','Weather_Risk','Piracy_Risk']:
    tmp[cat] = collapse_rare(tmp[cat], min_count=2).astype(str)


In [ ]:
# 2) Scale numerics; drop Sum_Insured if almost collinear with Premium
corr_ps = tmp[['Premium','Sum_Insured']].corr().iloc[0,1]
drop_sum_insured = abs(corr_ps) > 0.95

numz = ['Vessel_Age','Premium'] + ([] if drop_sum_insured else ['Sum_Insured'])
scaler = StandardScaler()
tmp[[c+'_z' for c in numz]] = scaler.fit_transform(tmp[numz])


In [ ]:
# 3) AUTO-DROP fixed effects with zero within-zone variance (non-identifiable with RI)
def has_within_group_variation(s, g):
    # True if at least one group shows >1 unique value (for numeric) or >1 level (for cat)
    by = s.groupby(g)
    # for numerics
    if np.issubdtype(s.dropna().dtype, np.number):
        return any(by.nunique() > 1)
    # for categoricals/objects
    return any(by.nunique() > 1)

fixed_blocks = {
    'NUM': ['Vessel_Age_z','Premium_z'] + ([] if drop_sum_insured else ['Sum_Insured_z']),
    'VT' : ['C(Vessel_Type)'],
    'IS' : ['C(Inspection_Status)'],
    'WR' : ['C(Weather_Risk)'],
    'PR' : ['C(Piracy_Risk)'],
}


In [ ]:

# Filter out blocks with zero within-zone variation
kept_blocks = []
for tag, terms in fixed_blocks.items():
    keep = True
    for t in terms:
        if t.startswith('C('):  # categorical in patsy form
            col = t[2:-1]
            if not has_within_group_variation(tmp[col], tmp['Operating_Zone']):
                keep = False
                break
        else:  # numeric z
            base = t.replace('_z','')
            if not has_within_group_variation(tmp[base], tmp['Operating_Zone']):
                keep = False
                break
    if keep:
        kept_blocks.append(tag)

print("Blocks with within-zone variation:", kept_blocks)


In [ ]:
# 4) Incremental fit: start minimal, add blocks one by one; skip if singular
def try_mixed(rhs_terms):
    formula = "log_claim ~ " + " + ".join(rhs_terms)
    try:
        fit = sm.MixedLM.from_formula(formula, groups='Operating_Zone',
                                      re_formula='1', data=tmp).fit(reml=True, method='lbfgs', maxiter=200)
        return fit
    except Exception as e:
        return e

# minimal numeric core
rhs = fixed_blocks['NUM'].copy()
print("Trying minimal NUM model:", rhs)
fit_best = try_mixed(rhs)
if isinstance(fit_best, Exception):
    # last resort: drop to a single numeric (Vessel_Age_z) and re-try
    rhs = ['Vessel_Age_z']
    print("Fallback minimal model:", rhs)
    fit_best = try_mixed(rhs)



In [ ]:

# add categorical blocks if available & identifiable
for tag in ['VT','IS','WR','PR']:
    if tag in kept_blocks:
        candidate_rhs = rhs + fixed_blocks[tag]
        print("Trying to add block", tag, "->", candidate_rhs)
        fit_try = try_mixed(candidate_rhs)
        if not isinstance(fit_try, Exception):
            rhs, fit_best = candidate_rhs, fit_try
            print("Added", tag)
        else:
            print("Skipping", tag, "due to singularity / convergence:", fit_try)

print("\nFINAL FORMULA:", "log_claim ~ " + " + ".join(rhs))
print(fit_best.summary() if not isinstance(fit_best, Exception) else fit_best)



**Discussion prompts (record your notes):**
- Did the mixed model reduce AIC vs OLS?
- Which fixed effects remain significant after adding random intercepts?
- Why might **Operating_Zone** heterogeneity matter for hull insurance pricing or risk?



---
## Lab 2 – Frequentist vs. Bayesian Approaches

**Goal:** Compare estimates and interval interpretations across frequentist and Bayesian frameworks.

- **Frequentist**: We already fit using `statsmodels` MixedLM.
- **Bayesian**: Below is a **template** using `PyMC`/`Bambi`. This environment may not have these libraries preinstalled. The cell checks availability and, if missing, **skips execution** but leaves **ready-to-run code** for your environment.


In [ ]:
!pip install bambi --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 3.5 MB/s eta 0:00:00


In [ ]:
# ==== Lab 2: Frequentist vs Bayesian (Template + Fallback) ====
import importlib, sys

has_bambi = importlib.util.find_spec('bambi') is not None
has_pymc = importlib.util.find_spec('pymc') is not None

print('Has bambi:', has_bambi)
print('Has pymc:', has_pymc)


Has bambi: True
Has pymc: True


In [ ]:


if has_bambi and has_pymc:
    import bambi as bmb
    # Bayesian mixed model analogous to Lab 2
    # Using weakly-informative priors by default
    model_bayes = bmb.Model(
        'log_claim ~ Vessel_Age + Premium + Sum_Insured + 1 + (1 + Vessel_Age|Operating_Zone) + '
        'C(Vessel_Type) + C(Inspection_Status) + C(Weather_Risk) + C(Piracy_Risk)',
        data=work
    )
    idata = model_bayes.fit(draws=1000, chains=2, cores=1)
    print(idata)
    # Posterior summaries
    print(model_bayes.summary())
else:
    print('\nBayesian libraries not available in this runtime.')
    print('Template code is provided below for environments with PyMC/Bambi installed.')



### Template: Bayesian Mixed Model with PyMC (via Bambi)

```python
import bambi as bmb
model_bayes = bmb.Model(
    'log_claim ~ Vessel_Age + Premium + Sum_Insured + 1 + (1 + Vessel_Age|Operating_Zone) + '
    'C(Vessel_Type) + C(Inspection_Status) + C(Weather_Risk) + C(Piracy_Risk)',
    data=work
)
idata = model_bayes.fit(draws=2000, chains=4, cores=4)
model_bayes.summary()
# Compare: frequentist confidence intervals vs. Bayesian credible intervals.
```
